ROC curve

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
fpr_roberta, tpr_roberta, _ = roc_curve(y_test, y_pred_proba_roberta)

plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {auc(fpr_rf, tpr_rf):.2f})")
plt.plot(fpr_roberta, tpr_roberta, label=f"XLM-RoBERTa (AUC = {auc(fpr_roberta, tpr_roberta):.2f})")
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.grid(True)
plt.show()


Disagreement hitmap:Where the two models disagree (and who gets it right).

In [ ]:
import seaborn as sns
import pandas as pd

df_disagreement = pd.DataFrame({
    "true": y_test,
    "rf_pred": y_pred_rf,
    "roberta_pred": y_pred_roberta
})

conf_matrix = pd.crosstab(
    df_disagreement['rf_pred'],
    df_disagreement['roberta_pred'],
    rownames=["RF prediction"], colnames=["RoBERTa prediction"]
)

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues")
plt.title("Disagreement Matrix (Prediction Counts)")
plt.show()


Per-Publisher Hyperpartisan Prediction % (Barplot)

In [ ]:
# Combine results from both models
df_rf = consistent_df_rf.set_index("domain")
df_roberta = consistent_df_roberta.set_index("domain")
combined = pd.concat([df_rf["percentage_hyper"], df_roberta["percentage_hyper"]], axis=1)
combined.columns = ["Random Forest", "RoBERTa"]

# Filter top 10 publishers by article count
top_publishers = df_rf.sort_values("article_count", ascending=False).head(10).index
combined = combined.loc[top_publishers]

# Plot
combined.plot(kind="bar", figsize=(12, 6))
plt.ylabel("% Predicted Hyperpartisan")
plt.title("Top Publishers: Hyperpartisan % by Model")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()
